# 2. Limpieza de los datos

José Luis Aguilera Luzania

## Contenido

**2.1 Introducción**

- ¿Qué es la detección de noticias falsas? (*Fake news*)
- Objetivo de la libreta

**2.2 Limpieza de los datos**

- ¿Por qué limpiar los datos?
- Librerías
- Conjunto de datos
- Eliminar columnas innecesarias
- Procesar las columnas restantes

**2.3 Traducción de los datos**

- Traducir el nombre de las columnas
- Traducir los temas
- Traducir las categorías

**2.4 Guardar los datos**

- Guardar los datos.



## 2.1 Introducción

**¿Qué es la detección de noticias falsas? (*Fake News*)**
La detección de noticias falsas (Fake News) es la tarea de evaluar la veracidad de las afirmaciones en las noticias. Este es un problema crítico en el Procesamiento del Lenguaje Natural (PLN) porque tanto en medios de noticias tradicionales como en medios digitales las Fake News generan un gran impacto social y político en cada individuo. Por ejemplo, la exposición a las Fake News puede generar actitudes de ineficacia, alienación y cinismo hacia ciertos candidatos políticos (Balmas, 2014).

**Objetivo de la libreta**
El objetivo de esta libreta es limpiar los datos para evitar errores durante el análisis, el procesamiento y obtener solo los datos necesarios para las siguientes libretas.

## 2.2 Limpieza de los datos

**¿Por qué limpiar los datos?**

Durante la libreta anterior "*1. Introducción a los datos*" se observó casos donde una fuente se encontraba escrita de distintas formas, aumentando la cantidad de fuentes únicas reconocidas. Esto puede pasar con palabras en todos los textos, ya sea por un acento, mayúscula, minúscula, etc. escrita de manera diferente en alguna entrada. Con el fin de prevenir errores en análisis futuros de los datos y en su procesamiento antes de ser utilizados por algún método, los datos deben ser limpiados y como extra serán traducidos al español para evitar inconsistencias en el idioma.

**Librerías**
Manipulación de datos:
- `pandas`: Librería para manipular los datos de forma tabular.
- `unicode`: Librería para eliminar acentos de las palabras.
- `re`: Librería para utilizar expresiones regulares.


In [1]:
import pandas as pd
import unidecode
import re

**Conjunto de datos**
Los datos están divididos en dos conjuntos de datos en los archivos `train.xlsx` y `development.xlsx`, con 80% para entrenamiento y 20% para pruebas respectivamente.

Para limpiar los datos y ordenarlos para su uso posterior se unirán en un solo dataframe de pandas.

In [2]:
# Leer los datos.
df_entrenamiento = pd.read_excel('Datos/train.xlsx')
df_pruebas = pd.read_excel('Datos/development.xlsx')

# Combinar los datos.
df_datos = pd.concat([df_entrenamiento, df_pruebas])

# Información del dataframe.
print(f'Noticias, Columnas: {df_datos.shape}')
print(f'Columnas: {list(df_datos.columns)}')
print('Noticias verdaderas: {}'.format(df_datos.groupby('Category').size()[0]))
print('Noticias falsas: {}'    .format(df_datos.groupby('Category').size()[1]))

# Primeros elementos del dataframe.
df_datos.head()

Noticias, Columnas: (971, 7)
Columnas: ['Id', 'Category', 'Topic', 'Source', 'Headline', 'Text', 'Link']
Noticias verdaderas: 480
Noticias falsas: 491


,Id,Category,Topic,Source,Headline,Text,Link
0,1,Fake,Education,El Ruinaversal,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...","RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...",http://www.elruinaversal.com/2017/06/10/rae-in...
1,2,Fake,Education,Hay noticia,"La palabra ""haiga"", aceptada por la RAE","La palabra ""haiga"", aceptada por la RAE La Rea...",https://haynoticia.es/la-palabra-haiga-aceptad...
2,3,Fake,Education,El Ruinaversal,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,http://www.elruinaversal.com/2018/05/06/yordi-...
3,4,True,Education,EL UNIVERSAL,UNAM capacitará a maestros para aprobar prueba...,UNAM capacitará a maestros para aprobar prueba...,http://www.eluniversal.com.mx/articulo/nacion/...
4,5,Fake,Education,Lamula,pretenden aprobar libros escolares con conteni...,Alerta: pretenden aprobar libros escolares con...,https://redaccion.lamula.pe/2018/06/19/memoria...


**Eliminar las columnas `Id` y `Link`**

Los datos de la columna:
- `Id` son útiles para identificar cada una de las instancias de las noticias.
- `Link` son útiles para leer el artículo original o comprobar si la noticia existe.

en este caso, no son datos necesarios o útiles para el siguiente análisis.


In [3]:
# Eliminar las columnas.
df_datos = df_datos.drop(columns=['Id', 'Link'])

# Comprobar las columnas del dataframe.
print(f'Columnas: {list(df_datos.columns)}')

Columnas: ['Category', 'Topic', 'Source', 'Headline', 'Text']


**Procesar las columnas restantes**

Con el fin de agilizar el proceso de procesamiento, se definen dos funciones: `procesar_texto` y `procesar_columnas`.

- La función `procesar_textos` es la función `procesar_fuentes` de la libreta anterior, solo que renombrada para que se ajuste a su nuevo propósito.
- La función `procesar_columnas` aplicará la función `procesar_textos` a las columnas especificadas del conjunto de datos.

Como recordatorio la función `procesar_textos` realiza lo siguiente:
1. Convertir el texto en minúsculas.
2. Eliminar acentos.
3. Eliminar todo lo que no sea una palabra.


In [4]:
def procesar_texto(texto):
    """

    :param texto: cadena de texto.
    :return: Una cadena de texto formada solo por las palabras de la cadena original.
    """

    # 1. Convertir el texto en minúsculas.
    texto = texto.lower()

    # 2. Eliminar acentos.
    texto = unidecode.unidecode(texto)

    # 3. Quedarnos solo con las palabras.
    texto = re.findall(r'\w+', texto)
    texto = ' '.join(texto)

    return texto

def procesar_columnas(df, columnas, funcion_procesar):
    """

    :param df: dataframe con las columnas a procesar.
    :param columnas: lista de las columnas a procesar.
    :param funcion_procesar: función de procesamiento que se aplica a todas las columnas.
    :return: El dataframe con los datos de las columnas procesados.
    """

    for col in columnas:
        df[col] = df[col].apply(funcion_procesar)
    return df

# Lista de columnas a procesar.
columnas_a_procesar = ['Category', 'Topic', 'Source', 'Headline', 'Text']

# Procesamiento.
df_datos = procesar_columnas(df_datos, columnas_a_procesar, procesar_texto)
df_datos.head()

,Category,Topic,Source,Headline,Text
0,fake,education,el ruinaversal,rae incluira la palabra lady en el diccionario...,rae incluira la palabra lady en el diccionario...
1,fake,education,hay noticia,la palabra haiga aceptada por la rae,la palabra haiga aceptada por la rae la real a...
2,fake,education,el ruinaversal,yordi rosado escribira y disenara los nuevos l...,yordi rosado escribira y disenara los nuevos l...
3,true,education,el universal,unam capacitara a maestros para aprobar prueba...,unam capacitara a maestros para aprobar prueba...
4,fake,education,lamula,pretenden aprobar libros escolares con conteni...,alerta pretenden aprobar libros escolares con ...


## 2.3 Traducción de los datos

**Traducción de las columnas**

Las columnas se encuentran en inglés, pero el texto, título y fuente en español, para evitar inconsistencias se traducen todas las columnas.

In [5]:
columnas_dict = {
    'Category': 'categoria',
    'Topic':    'tema',
    'Source':   'fuente',
    'Headline': 'encabezado',
    'Text':     'texto'
}

# Renombrar las columnas.
df_datos.rename(columns=columnas_dict, inplace=True)

# Imprimir las columnas.
print(f'Columnas: {list(df_datos.columns)}')

Columnas: ['categoria', 'tema', 'fuente', 'encabezado', 'texto']


**Traducción de los temas**

Los temas también se traducen al español.

Como recordatorio los temas abarcados en las noticias son los siguientes:
1. Ciencia.
2. Deportes.
3. Economía.
4. Entretenimiento.
5. Educación.
6. Política.
7. Salud.
8. Seguridad.
9. Sociedad

In [6]:
def traducir_temas(df):
    df.loc[df['tema'] == 'economy'      , 'tema'] = 'economía'
    df.loc[df['tema'] == 'education'    , 'tema'] = 'educación'
    df.loc[df['tema'] == 'entertainment', 'tema'] = 'entretenimiento'
    df.loc[df['tema'] == 'health'       , 'tema'] = 'salud'
    df.loc[df['tema'] == 'politics'     , 'tema'] = 'política'
    df.loc[df['tema'] == 'science'      , 'tema'] = 'ciencia'
    df.loc[df['tema'] == 'security'     , 'tema'] = 'seguridad'
    df.loc[df['tema'] == 'society'      , 'tema'] = 'sociedad'
    df.loc[df['tema'] == 'sport'        , 'tema'] = 'deportes'

traducir_temas(df_datos)
df_datos.head()

,categoria,tema,fuente,encabezado,texto
0,fake,educación,el ruinaversal,rae incluira la palabra lady en el diccionario...,rae incluira la palabra lady en el diccionario...
1,fake,educación,hay noticia,la palabra haiga aceptada por la rae,la palabra haiga aceptada por la rae la real a...
2,fake,educación,el ruinaversal,yordi rosado escribira y disenara los nuevos l...,yordi rosado escribira y disenara los nuevos l...
3,true,educación,el universal,unam capacitara a maestros para aprobar prueba...,unam capacitara a maestros para aprobar prueba...
4,fake,educación,lamula,pretenden aprobar libros escolares con conteni...,alerta pretenden aprobar libros escolares con ...


*Nota*: Los temas se traducen al español para un análisis exploratorio consistente, sin embargo, esto solo es posible debido al número limitado de temas. En caso de existir más temas no se recomienda traducirlos uno a uno como se hizo aquí.


**Traducir categorías**

Las categorías no son traducidas, en su lugar se convierten en valores numéricos para tener un mejor control durante el análisis y el procesamiento de los datos.

In [7]:
def traducir_categorias(df):
    df.loc[df['categoria'] == 'fake', 'categoria'] = '0'
    df.loc[df['categoria'] == 'true', 'categoria'] = '1'

traducir_categorias(df_datos)
df_datos.head()

,categoria,tema,fuente,encabezado,texto
0,0,educación,el ruinaversal,rae incluira la palabra lady en el diccionario...,rae incluira la palabra lady en el diccionario...
1,0,educación,hay noticia,la palabra haiga aceptada por la rae,la palabra haiga aceptada por la rae la real a...
2,0,educación,el ruinaversal,yordi rosado escribira y disenara los nuevos l...,yordi rosado escribira y disenara los nuevos l...
3,1,educación,el universal,unam capacitara a maestros para aprobar prueba...,unam capacitara a maestros para aprobar prueba...
4,0,educación,lamula,pretenden aprobar libros escolares con conteni...,alerta pretenden aprobar libros escolares con ...


## 2.4 Guardar los datos

Ahora los datos se encuentran procesados y traducidos al español, es necesario guardarlos en un archivo para poder utilizarlo en las libretas posteriores. Los datos son guardados en un archivo llamado `datos` con extensión `.csv`.

In [8]:
df_datos.to_csv('Datos/datos.csv', index=False)